In [ ]:
from typing import List
import uproot
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import matplotlib.ticker as mticker
import os

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))
plt.rcParams.update({"font.size": 20})

In [ ]:
MAIN_DIR = "../../../../"

plot_dir = f"{MAIN_DIR}/plots/BiasTest/23Jul10Res"
_ = os.system(f"mkdir -p {plot_dir}")

In [ ]:
cards_dir = "f_tests/23May2/nTF1_2_nTF2_1"
biases = [0.0, 0.15, 0.3, 1.0]
r_dict = {}

for bias in biases:
    file = uproot.concatenate(
        f"/uscms/home/rkansal/hhcombine/cards/{cards_dir}/higgsCombinebias{bias}.FitDiagnostics.mH125.*.root"
    )

    r_dict[bias] = {
        "r": np.array(file["trackedParam_r"])[::4],
        "rerr": np.array(file["trackedError_r"])[::4],
    }

In [ ]:
from scipy import stats

xrange = 4
x = np.linspace(-xrange, xrange, 101)

fig, ax = plt.subplots(len(biases), 1, figsize=(12, 40))

for i, bias in enumerate(biases):
    r = r_dict[bias]["r"]
    rerr = r_dict[bias]["rerr"]
    reldiff = (r - bias) / rerr
    reldiff = reldiff[(reldiff > -4) * (reldiff < 4)]
    # reldiff = (r - bias)
    mu, sigma = np.mean(reldiff), np.std(reldiff)
    ax[i].hist(reldiff, np.linspace(-xrange, xrange, 21), histtype="step")
    ax[i].plot(
        x,
        stats.norm.pdf(x, loc=mu, scale=sigma) * len(r) * xrange / 10,
        label=rf"$\mu = {mu:.2f}, \sigma = {sigma:.2f}$",
    )
    ax[i].set_xlabel(rf"$\frac{{\hat{{r}} - {bias}}}{{\Delta \hat r}}$")
    ax[i].set_ylabel("Number of toys")
    ax[i].set_title(f"r = {bias}")
    ax[i].legend()

plt.show()

In [ ]:
np.mean()

In [ ]:
np.std((r - bias) / rerr)

In [ ]:
res.params

In [ ]:
r_dict[0.3]

In [ ]:
def plot_tests(
    data_ts: float,
    toy_ts: np.ndarray,
    name: str,
    title: str = None,
    bins: int = 15,
    fit: str = None,
    fdof2: int = None,
):
    plot_max = max(np.max(toy_ts), data_ts)
    # plot_max = max(np.max(toy_ts), data_ts) if fit != "chi2" else 200
    # plot_min = min(np.min(toy_ts), data_ts, 0)
    plot_min = 0
    pval = p_value(data_ts, toy_ts)

    plt.figure(figsize=(12, 8))
    h = plt.hist(
        toy_ts,
        np.linspace(plot_min, plot_max, bins + 1),
        color="#8C8C8C",
        histtype="step",
        label=f"{len(toy_ts)} Toys",
    )
    plt.axvline(data_ts, color="#FF502E", linestyle=":", label=rf"Data ($p$-value = {pval:.2f})")

    if fit is not None:
        x = np.linspace(plot_min + 0.01, plot_max, 100)

        if fit == "chi2":
            res = stats.fit(stats.chi2, toy_ts, [(0, 200)])
            pdf = stats.chi2.pdf(x, res.params.df)
            label = rf"$\chi^2_{{DoF = {res.params.df:.2f}}}$ Fit"
        elif fit == "f":
            pdf = stats.f.pdf(x, 1, fdof2)
            label = rf"$F-dist_{{DoF = (1, {fdof2})}}$"
        else:
            raise ValueError("Invalid fit")

        plt.plot(
            x,
            pdf * (np.max(h[0]) / np.max(pdf)),
            color="#1f78b4",
            linestyle="--",
            # alpha=0.6,
            label=label,
        )

    hep.cms.label(
        "Work in Progress",
        data=True,
        lumi=138,
        year=None,
    )

    _ = plt.legend()
    plt.title(title)
    plt.ylabel("Number of Toys")
    plt.xlabel("Test Statistics")

    plt.savefig(f"{plot_dir}/{name}.pdf", bbox_inches="tight")

Nonresonant

In [ ]:
o1 = 0  # order being tested
tlabel = f"{o1}"

data_ts, toy_ts = test_statistics[tlabel]["data"][tlabel], test_statistics[tlabel]["toys"][tlabel]
plot_tests(data_ts, toy_ts, "gof" + tlabel, fit="chi2", bins=20)

ord1 = 1
tflabel = f"{ord1}"
data_ts, toy_ts = pval = (
    test_statistics[tlabel]["fdata"][tflabel],
    test_statistics[tlabel]["ftoys"][tflabel],
)
plot_tests(data_ts, toy_ts, f"f{tlabel}_{tflabel}", title=f"{o1} vs. {ord1}")

Resonant

In [ ]:
o1, o2 = 2, 0  # order being tested
tlabel = f"{o1}{o2}"

data_ts, toy_ts = test_statistics[tlabel]["data"][tlabel], test_statistics[tlabel]["toys"][tlabel]
plot_tests(data_ts, toy_ts, "gof" + tlabel, fit="chi2", bins=20)

for ord1, ord2 in [[o1 + 1, o2], [o1, o2 + 1]]:
    tflabel = f"{ord1}{ord2}"
    data_ts, toy_ts = pval = (
        test_statistics[tlabel]["fdata"][tflabel],
        test_statistics[tlabel]["ftoys"][tflabel],
    )
    plot_tests(data_ts, toy_ts, f"f{tlabel}_{tflabel}", title=f"({o1}, {o2}) vs. ({ord1}, {ord2})")